In [1]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
from nltk.corpus import stopwords
import string
import os
import json
from nltk.corpus import wordnet
import glob
import csv
import pandas as pd

# Thematic vocabulary concepts

In [11]:
# merge and compute mean value of cvalue and tfidfcm
def merg_cval_and_tfidfcm_terms(root):
    #for file in root:
    files = glob.glob(root + '/*.csv',recursive = True) 
    print(files)
    with open(files[0],newline='') as f:
        r = csv.reader(f)
        data = [line for line in r]
        print(len(data))
    with open(files[1],newline='') as f:
        r1 = csv.reader(f)
        data1 = [line for line in r1]
        print(len(data1))
    with open(root+'/cv_tf.csv','a+',newline='') as f:
        w = csv.writer(f)
        w.writerow(['terms', 'in_umls', 'rank'])
        w.writerows(data)
        w.writerows(data1)
             
# rootPath contains all f_dir files
def compute_mean_val_of_merged_terms(f_dir):
    files = glob.glob(f_dir + '/*cv_tf.csv',recursive = True) 
    #for file in Path(f_dir).rglob('*cv_tf.csv'):
    root, filenam = os.path.split(files[0])
    terms = pd.read_csv(files[0], sep=',|;', engine='python')
    #print(len(terms))
    #filename = file.name
    filename = filenam.split('.csv')[0]
    #terms = pd.read_csv(file, sep=",")
    df1 = terms.groupby(['terms'])['rank'].agg([np.mean])
    #print(df1)
    df1 = df1.sort_values(["mean"], ascending=False)
    print('final-len', len(df1))
    df1.to_csv(root+"/"+filename+"_mean.csv", sep='\t')

In [12]:
# read kwds list
def read_kwd(txtfile):
    f = open(txtfile)
    content = f.read()
    kw_list = (content.lower()).split('\n')
    return kw_list

In [13]:
# terms semantic similarity evaluation
def eval_terms(corpus,queries,top_k):
    embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    D = {}
    for query in queries:
        D[query] = []
        query_embedding = embedder.encode(query, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
        cos_scores = cos_scores.cpu()

        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

        for idx in top_results[0:top_k]:
            D[query].append( float('%.4f' % (cos_scores[idx])) )
    for val_ in D:
        D[val_]= sum(D[val_])/len(D[val_])

    return D

In [14]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [15]:
# def finalRank_from_BertAnd_Biotex(f_biotex, f_bert):
#     d_biotex, d_bert = {}, {}
#     df_biotex = pd.read_csv(f_biotex, sep='\t|,|;', engine='python')
#     df_bert = pd.read_csv(f_bert, sep='\t|,|;', engine='python')

#     for i in range(len(df_biotex['terms'])):
#         d_biotex[df_biotex['terms'][i]] = 1/(i+1)

#     for i in range(len(df_bert['terms'])):
#         d_bert[df_bert['terms'][i]] = df_bert['rank'][i]
#     D = {}
#     for k in d_biotex:
#         if k in d_bert:
#             Dic[k] = NormalizeData(a[k]+b[k]/2)
        
#     return D#d_biotex, d_bert

In [16]:
# main function for similarity measure
def score_mrgd_ts_and_expert_ts_sim(t_file, kwd_file, n_first):
    d_biotex = {}
    files = glob.glob(t_file + '/*mean.csv',recursive = True) 
    root, filenam = os.path.split(files[0])
    filenam = filenam.split('.csv')[0]

    kw_list = read_kwd(kwd_file)
    kw_list = kw_list[:-1]

    terms = pd.read_csv(files[0], sep='\t', engine='python')

    df = terms['terms']
    df = np.array(df)
    
    df = [word for word in df if (word not in stopwords.words('french'))]
    df = [word for word in df if word]
    d_bert = eval_terms(kw_list ,df[:1000],n_first)
    
    ########### use to score with Biotex data pertinence range
#     for i in range(len(terms['terms'])):
#         d_biotex[terms['terms'][i]] = 1/(i+1)
#     print (d_biotex)
#     D = {}
#     for k in d_biotex:
#         if k in d_bert:
#             D[k] = NormalizeData(d_biotex[k]+d_bert[k]/2)
    ##########
    #print(D)
    df = pd.DataFrame(list(d_bert.items()), columns=['terms','rank'])
    final_df = df.sort_values(by=['rank'], ascending=False)

    final_df.to_csv(root+'/'+filenam+'_Bert.csv')

In [17]:
def compute_best_terms(biotex_out, expert_concept, n_first):
    #root, filenam = os.path.split(biotex_out)
    merg_cval_and_tfidfcm_terms(biotex_out)
    compute_mean_val_of_merged_terms(biotex_out)
    data = score_mrgd_ts_and_expert_ts_sim(biotex_out, expert_concept,n_first)
    return data

In [19]:
# biotex_out = "/home/rodrique/Bureau/Jupyter-notebook/Protocole_3M/Protocole_aidmoit/biotex_out_"
# expert_concept = '/home/rodrique/Bureau/Jupyter-notebook/Protocole_3M/Protocole_aidmoit/expert_concept/agriculture.txt'
# n_first = 10
# data = compute_best_terms(biotex_out, expert_concept, n_first) # data is in dic format 

['/home/rodrique/Bureau/Jupyter-notebook/Protocole_3M/Protocole_aidmoit/biotex_out_/tf_agri.csv', '/home/rodrique/Bureau/Jupyter-notebook/Protocole_3M/Protocole_aidmoit/biotex_out_/cv_agri.csv']
127071
127087


<IPython.core.display.Javascript object>

final-len 127124


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# biotex_out = "/home/rodrique/Bureau/Protocole/Corpus_experiment/corpus/agri"
# expert_concept = '/home/rodrique/Bureau/Jupyter-notebook/Protocole_3M/doc_collect_adapted/keywords/agriculture.txt'
# data = compute_best_terms(biotex_out, expert_concept, 10)

In [37]:
##usage
# definir le chemin d'accès du fichier contenant l'ensemble des termes
# indiquer aussi lle chemin du fichier de termes experts de la thematic concernee
# indiquer la taille ou nombre de termes à prendre en compte pour le calcule de similarité -n_first-

#score_mrgd_ts_and_expert_ts_sim(merged_file, kwd_file,n_first)

# compute doc similarity with extend vocabulary

In [38]:
# Use this if corpus is saved with corresponding source data
# dic is formated as: {corpus_name:corpus_data}
def load_json(file):
    with open(file) as json_file:
        data = json.load(json_file)
    return data


def save_json(dic,des):
    with open(des, 'w') as fp:
        json.dump(dic, fp)


def _format(kwd_list):
    nl_ = []
    for t in kwd_list:
        tmp = t.split(' ')
        tmp = [i for i in tmp if i]
        for tm in tmp:
            nl_.append(tm)
    #print(len(nl_))
    nl_ = [word for word in nl_ if (word not in stopwords.words('french'))]
    nl_ = np.unique(nl_)
    #x = " ".join(k for k in nl_)
    return nl_ #x


# read keywords file and return each one as sentence
def read_kwd_(txtfile):
    f = open(txtfile)
    content = f.read()
    kw_list = (content.lower()).split('\n')
    kw_list = _format(kw_list)
    return kw_list


# termes similarity scoring with DistilBert
# corpus ==> list of extracted terms from the corpus with BioTex
# return a dictionary, keys =>  extracted terms, and values => mean of the whole similarity score!
# top_k => k
def measure_(corpus,query):
    embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
    # Corpus with example sentences
    corpus_embeddings = embedder.encode(query, convert_to_tensor=True)
    #D = {}
    query_embedding = embedder.encode(corpus, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()
    return cos_scores


#root = "/home/rodrique/Bureau/Protocole/test_collecte/tst2"
def score_doc_sim(extend_voc_concept, doc):
    #kwd_list = read_kwd_(extend_voc_concept)
    kwd_cp = " ".join(k for k in extend_voc_concept[:1000]) #[:1000]
    #print(kwd_cp)
    score = measure_(doc["text"],kwd_cp)
    doc["pertinence"] = round(score.tolist()[0], 2)
    return doc

In [39]:
def get_synonym(voc_concept):
    #Creating a list
    synonyms = []
    for syn in wordnet.synsets(voc_concept,lang='fra'):#, lang='fra'
        for lm in syn.lemmas('fra'):
            synonyms.append(lm.name())#adding into synonyms
    synonyms = set(synonyms)
    return synonyms


# List data to csv file
def write_list_to_txt(terms_list, outfile):
    """Write the list to csv file."""
    with open(outfile, 'w') as f:
        for item in terms_list:
            f.write("%s\n" % item)
    return outfile

# recoder pour prendre en compte un fichier csv, la sortie du vocabulaire de concepts final
def generate_new_terms(voc_concept):
    data = read_kwd(voc_concept)
    root, filname = os.path.split(voc_concept)
    filname = filname.split('.txt')[0]
    L = []
    for t in data:
        t = t.split(' ')
        if t.__len__() == 1:
            syn = [get_synonym(a) for a in t]
            for v in syn[0]:
                L.append(v)
        if t.__len__() > 1:
            syn = [get_synonym(a) for a in t]
            #print(syn)
            for s in syn[0]:
                test = " ".join(t[1:])
                w = s.lower() + ' ' + test
                #print('zzz', w)
                L.append(w)
    #print(L.__len__())
    L = set(L)
    #print(L.__len__())
    #write_list_to_txt(L,root + '/'+ filname+'_syno.txt')
    return list(L)

In [40]:
# voc_concept = '/home/rodrique/Bureau/Jupyter-notebook/Protocole_3M/doc_collect_adapted/keywords/agriculture.txt'
# len(list(generate_new_terms(voc_concept)))

In [41]:
# usage
#data = '/home/rodrique/PycharmProjects/Corpus/config/keywords_1500/urbanisation.txt'
#generate_new_terms(data)

In [42]:
def compute_best_doc(voc_concept, doc):
    extent_voc_concept = generate_new_terms(voc_concept)
    doc = score_doc_sim(extent_voc_concept, doc)
    return doc

In [43]:
# voc_concept = '/home/rodrique/Bureau/Jupyter-notebook/Protocole_3M/doc_collect_adapted/keywords/agriculture.txt'
# doc = {'text' : "C'est particulièrement net quand les idées ou initiatives à promouvoir viennent en décalage ou en opposition avec les formes dominantes du système existant. Par exemple, développer les circuits courts ou les grossistes en opposition à la grande distribution, favoriser l'installation diversifiée dans un contexte de viticulture spécialisée et de difficulté d'installation, savoir quelle forme d'agriculture promouvoir en fonction des multiples intérêts sur le territoire, identifier les réserves d'emplois dans le système alimentaire et soutenir la création d'emplois qualifiés. Tous ces sujets soulèvent des incertitudes ou des controverses sur ce qu'il est souhaitable et pertinent de faire, ici et maintenant."}
# compute_best_doc(voc_concept,doc)